In [ ]:
import warnings
import time

import sacrebleu
import itertools

from transformers import TFMarianMTModel, MarianTokenizer

warnings.filterwarnings("ignore")

In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-uk"
tokenizer = MarianTokenizer.from_pretrained(model_name)

model = TFMarianMTModel.from_pretrained(model_name)

model.summary()

In [ ]:
# Define input text
input_text = "Hello, how have You been?"

start_time = time.time()

# Tokenize input text
input_ids = tokenizer([input_text], return_tensors="tf", padding=True)

# Translate input text
translated_tokens = model.generate(input_ids["input_ids"])

# Decode translated tokens
translated_text = tokenizer.decode(
    translated_tokens.numpy()[0], skip_special_tokens=True
)

print("--- %s seconds ---" % (time.time() - start_time))

print(f"Input Text: {input_text}.")
print(f"Translated Text: {translated_text}.")

In [ ]:
# Compute scores
references = [input_text]
hypotheses = [translated_text]

bleu_score = sacrebleu.corpus_bleu(hypotheses, [references]).score

chrf_score = sacrebleu.corpus_chrf(hypotheses, [references]).score

ter_score = sacrebleu.corpus_ter(hypotheses, [references]).score

print(f"BLEU Score: {bleu_score:.2f}.")
print(f"chrF Score: {chrf_score:.2f}.")
print(f"TER Score: {ter_score:.2f}.")

In [ ]:
config = model.config

config

In [ ]:
# Define the model name
model_name = "Helsinki-NLP/opus-mt-en-uk"

# Load the tokenizer and model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = TFMarianMTModel.from_pretrained(model_name)

# Define input text and reference translation
input_text = "Hello, how have you been?"
reference_translation = "Привіт, як справи?"

# Define candidate configurations to try
configurations = [
    {"max_length": 128, "num_beams": 4},
    {"max_length": 256, "num_beams": 8},
    {"max_length": 512, "num_beams": 16},
]

# Evaluate each configuration and choose the one with the highest BLEU score
best_bleu_score = 0
best_configuration = None

for config in configurations:
    # Set model configuration
    model.config = model.config.from_dict(config)

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors="tf")

    # Translate input text
    translated_tokens = model.generate(
        input_ids, max_length=config["max_length"], num_beams=config["num_beams"]
    )

    # Decode translated tokens
    translated_text = tokenizer.decode(
        translated_tokens.numpy()[0], skip_special_tokens=True
    )

    # Compute BLEU score
    hypotheses = [translated_text]
    bleu_score = sacrebleu.corpus_bleu(hypotheses, [[reference_translation]]).score

    # Update best configuration if BLEU score is higher
    if bleu_score > best_bleu_score:
        best_bleu_score = bleu_score
        best_configuration = config

# Print the best configuration and BLEU score
print("Best Configuration:", best_configuration)
print(f"Best BLEU Score: {best_bleu_score}")

In [ ]:
# Define candidate parameter values
max_lengths = [128, 256, 512]
num_beams_list = [4, 8, 16]
num_hidden_layers_list = [2, 4, 6]
# max_position_embeddings_list = [
#     128,
#     256,
#     512,
# ]  # Define different values for max_position_embeddings
# use_cache_list = [True, False]  # Define different values for use_cache

# Generate all possible combinations of parameter values
configurations = list(
    itertools.product(
        max_lengths,
        num_beams_list,
        num_hidden_layers_list,
        # max_position_embeddings_list,
        # use_cache_list,
    )
)

# Evaluate each configuration and choose the one with the highest BLEU score
best_bleu_score = 0
best_configuration = None
configuration_amount = 0

for config in configurations:
    configuration_amount += 1
    # max_length, num_beams, num_hidden_layers, max_pos_emb, cache = config
    max_length, num_beams, num_hidden_layers = config

    current_configuration = config
    print(f"Current configuration: {current_configuration}")

    # Set model configuration
    model.config = model.config.from_dict(
        {
            "max_length": max_length,
            "num_beams": num_beams,
            "num_hidden_layers": num_hidden_layers,
            # "max_position_embeddings": max_pos_emb,
            # "use_cache": cache,
        }
    )

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors="tf")

    start_time = time.time()

    # Translate input text
    translated_tokens = model.generate(
        input_ids, max_length=max_length, num_beams=num_beams
    )

    # Decode translated tokens
    translated_text = tokenizer.decode(
        translated_tokens.numpy()[0], skip_special_tokens=True
    )

    translation_time = time.time() - start_time
    print(f"translation_time: {translation_time:.2f}.")

    # Compute BLEU score
    hypotheses = [translated_text]
    bleu_score = sacrebleu.corpus_bleu(hypotheses, [[reference_translation]]).score

    # Update best configuration if BLEU score is higher
    if bleu_score > best_bleu_score:
        best_bleu_score = bleu_score
        best_configuration = {
            "max_length": max_length,
            "num_beams": num_beams,
            "num_hidden_layers": num_hidden_layers,
            # "max_position_embeddings": max_pos_emb,
            # "use_cache": cache,
        }
        time_of_best_translation = translation_time

In [ ]:
# Print the best configuration and BLEU score
print(f"Configuration amount: {configuration_amount}.")
print("Best Configuration:", best_configuration)
print(f"Best BLEU Score: {best_bleu_score:.2f}.")
print(f"Time: {time_of_best_translation:.2f} seconds.")